In [ ]:
import os
import copy
from collections import OrderedDict
from tabulate import tabulate
import numpy as np
import math
import matplotlib.pyplot as plt
import flopy
import flopy.plot.styles as styles
from dis2disu import Dis2Disu

In [ ]:
def set_up_dis_grid(nlay_chan, ncol, Lx, theta, domain):

    # grid parameters intended to remain fixed
    nrow = 1                        # model is one row deep "into the page"
    delc = 1.                       # model has unit width "into the page"
    
    # computed grid parameters
    if domain:
        nlay_dom_upper = nlay_chan                      # number of layers in upper domain
    else:
        nlay_dom_upper = 0
    nlay_dom_lower = nlay_dom_upper                     # number of layers in lower domain
    nlay = nlay_chan + nlay_dom_upper + nlay_dom_lower  # total number of layers
    delr = float(Lx / ncol)                             # horizontal cell size
    delz_chan = delr                                    # vertical cell size in channel
    zoffset = delr * math.tan(theta * math.pi / 180.)   # vertical offset between cells in channel
    zthick = nlay_chan * delz_chan                      # vertical thickness of channel
    zspan = (ncol - 1) * zoffset + zthick               # total vertical span of channel
    zthick_dom_ll = zthick                              # vertical thickness of lower domain at left boundary
    zthick_dom_ur = zthick                              # vertical thickness of upper domain at right boundary

    icell_chan_first = nlay_dom_upper * ncol                   # zero-based number of first cell in channel
    icell_chan_last = icell_chan_first + nlay_chan * ncol - 1  # zero-based number of last cell in channel
    icelltype = np.zeros((nlay * nrow * ncol))                 # array that is 1 for cells in channel, 0 for cells in domain
    for i in range(icell_chan_first, icell_chan_last + 1):
        icelltype[i] = 1

    # set tops and bottoms; midpoints of cell tops/bottoms fall along
    # the "true" top/bottom channel boundaries
    botm = np.empty((nlay, nrow, ncol))
    top_chan = zthick + 0.5 * zoffset \
        + np.linspace(0., (ncol - 1) * zoffset, ncol)
    top_chan = top_chan.reshape((nrow, ncol))           # top of channel (without domain)
    if domain:
        # upper domain
        Lz = zthick_dom_ll + zspan + zthick_dom_ur      # total height of model
        top_chan += zthick_dom_ll                       # add left-boundary thickness of lower domain to top of channel
        top = np.ones((nrow, ncol)) * Lz                # top of model
        dz = (top - top_chan) / nlay_dom_upper          # vertical cell size varies by column in domain
        botm[0] = top - dz                              # bottom of first layer is calculated using top
        for klay in range(1, nlay_dom_upper):
            botm[klay] = botm[klay - 1] - dz
        # prepare to continue with channel discretization
        klaycontinue = nlay_dom_upper
    else:
        # initialize channel discretization
        Lz = zspan                                     # total height of model
        top = top_chan                                 # top of model is top of channel
        botm[0] = top - delz_chan                      # bottom of first layer is calculated using top
        # prepare for channel discretization
        klaycontinue = 1
    # channel
    for klay in range(klaycontinue, nlay_dom_upper + nlay_chan):
        botm[klay] = botm[klay - 1] - delz_chan
    if domain:
        # lower domain
        dz = (top_chan - zthick) / nlay_dom_lower     # vertical cell size varies by column in domain
        for klay in range(nlay_dom_upper + nlay_chan, nlay):
            botm[klay] = botm[klay - 1] - dz

    delr = delr * np.ones(ncol, dtype=float)
    delc = delc * np.ones(nrow, dtype=float)
    mgs = flopy.discretization.StructuredGrid(delr=delr, delc=delc, top=top, botm=botm)
    temp1 = top - botm[0]
    temp1 = temp1.reshape(1,1,11)
    temp2 = botm[0:nlay-1] - botm[1:nlay]
    print('Lz = ',Lz)
    print('zspan = ', zspan)
    thick_arr = np.vstack((temp1.reshape(1,1,11),temp2))
    return(mgs, nlay_dom_upper, icelltype, delz_chan, zspan, zthick, zthick_dom_ll, thick_arr)

In [ ]:
def set_up_cond(k_dom):

    # set conductivities
    nlay = mgs.nlay
    nrow = mgs.nrow
    ncol = mgs.ncol
    cond = np.ones((nlay, nrow, ncol))                  # unit conductivity in channel, set as default here
    if domain:
        # upper domain
        for klay in range(nlay_dom_upper):
            cond[klay] = k_dom
        # lower domain
        for klay in range(nlay_dom_upper + nlay_chan, nlay):
            cond[klay] = k_dom
    
    return (cond)

In [ ]:
# function that builds parameters (ordered) dictionary from scenario options
def build_parameters(options, *args, ioption=0):
    if ioption == 0:
        # initial call is without args, so set things up
        noptions = len(options)
        name = [""] * (noptions + 1)
        name[0] = "disu"
        if domain:
            name[0] = name[0] + "-d"
        parameters = OrderedDict()
        pdict = OrderedDict()
        args = (name, pdict, parameters)
    else:
        # after initial call, args is passed, so extract things from it
        args = args[0]
        name, pdict, parameters = args
    # loop over option values
    for optionval in [False, True]:
        # update parameter sub-dictionary and scenario name array
        pdict[options[ioption]["optionname"]] = optionval
        name[ioption + 1] = name[ioption] + options[ioption]["optionstrings"][optionval]
        if ioption == len(options) - 1:
            # last option, so copy parameter sub-dictionary to parameters with scenario name as key
            parameters[name[ioption + 1]] = copy.deepcopy(pdict)
        else:
            # call this routine recursively for the next option
            ioptionnext = ioption + 1
            parameters = build_parameters(options, args, ioption=ioptionnext)
    return parameters

In [ ]:
def calculate_head_analyt():
    global head_analyt

    # Analytical head a la Bardot et al (2022),
    # assuming a head gradient of magnitude -1.
    # axial flow: head = - cos(theta) * x - sin(theta) * z
    # For K = 1., these expressions give unit flux.
    
    nlay = mgs.nlay
    delr = mgs.delr
    top = mgs.top
    botm = mgs.botm

    thetarad = theta * math.pi / 180.
    sintheta = math.sin(thetarad)
    costheta = math.cos(thetarad)
    hgradx = -costheta
    hgradz = -sintheta
    if domain:
        hgradx_dom = hgradx
        hgradz_dom = hgradz

    head_analyt = []
    if domain:
        # upper domain
        xref2 = 0.
        zref2 = 0.
        href2 = 0.
        xref1 = 0.
        zref1 = zthick_dom_ll
        href1 = href2 + hgradx_dom * (xref1 - xref2) + hgradz_dom * (zref1 - zref2)
        xref0 = 0.
        zref0 = zref1 + zthick
        href0 =  href1 + hgradx * (xref0 - xref1) + hgradz * (zref0 - zref1)
        for klay in range(0, nlay_dom_upper):
            for jcol in range(ncol):
                xc = float(jcol + 0.5) * delr[0]
                xcrel = xc - xref0
                if klay == 0:
                    ztop = top[0][jcol]
                else:
                    ztop = botm[klay - 1][0][jcol]
                zc = 0.5 * (ztop + botm[klay][0][jcol])
                zcrel = zc - zref0
                hc = href0 + hgradx_dom * xcrel + hgradz_dom * zcrel
                head_analyt.append(hc)
    else:
        xref1 = 0.
        zref1 = 0.
        href1 = 0.
    # channel
    for klay in range(nlay_dom_upper, nlay_dom_upper + nlay_chan):
        for jcol in range(ncol):
            xc = float(jcol + 0.5) * delr[0]
            xcrel = xc - xref1
            if klay == 0:
                ztop = top[0][jcol]
            else:
                ztop = botm[klay - 1][0][jcol]
            zc = 0.5 * (ztop + botm[klay][0][jcol])
            zcrel = zc - zref1
            hc = href1 + hgradx * xcrel + hgradz * zcrel
            head_analyt.append(hc)
    if domain:
        # lower domain
        for klay in range(nlay_dom_upper + nlay_chan, nlay):
            for jcol in range(ncol):
                xc = float(jcol + 0.5) * delr[0]
                xcrel = xc - xref2
                if klay == 0:
                    ztop = top[0][jcol]
                else:
                    ztop = botm[klay - 1][0][jcol]
                zc = 0.5 * (ztop + botm[klay][0][jcol])
                zcrel = zc - zref2
                hc = href2 + hgradx_dom * xcrel + hgradz_dom * zcrel
                head_analyt.append(hc)

    return

In [ ]:
def convert_to_disu(mgs, idx, dztol):
    global d2d
    
    delr = mgs.delr
    delc = mgs.delc
    top = mgs.top
    botm = mgs.botm
    key = list(parameters.keys())[idx]
    staggered = parameters[key]['staggered']
    d2d = Dis2Disu(delr, delc, top, botm, staggered=staggered, dztol=dztol)
    
    return (d2d)

In [1]:
def build_model(sim_name, xt3d, staggered):
    
    nlay = mgs.nlay
    delr = mgs.delr
    delc = mgs.delc
    top = mgs.top
    botm = mgs.botm
    
    calculate_head_analyt()
    
    sim_ws = os.path.join(ws, sim_name)
    sim = flopy.mf6.MFSimulation(sim_name=sim_name, sim_ws=sim_ws,
                                 exe_name='../exe/mf6.exe')
                                 #exe_name='../exe/mf6_xt3d_het.exe')  # kluge
    
    tdis = flopy.mf6.ModflowTdis(sim)
    ims = flopy.mf6.ModflowIms(sim, linear_acceleration='bicgstab',
                               #inner_dvclose=1e-12,
                               #inner_rclose=1e-12,
                              )
    gwf = flopy.mf6.ModflowGwf(sim, modelname=sim_name,
                               save_flows=True, print_flows=True)
    disu = flopy.mf6.ModflowGwfdisu(gwf, **d2d.get_gridprops_disu6())
    ic = flopy.mf6.ModflowGwfic(gwf, strt=head_analyt)
    
    if anisotropic:
        npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True,
                                      xt3doptions=xt3d, k=cond, k22 = cond, k33 = cond/ratio,
                                      angle1=0., angle2=dip, angle3=0.)
    else:
        npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True,
                                      xt3doptions=xt3d, k=cond, k22 = cond, k33 = cond,
                                      angle1=0., angle2=0., angle3=0.)

    # Set boundary heads to analytical values
    spd = []
    # specify heads along left and right sides of model
    for klay in range(nlay):
        icleft = klay*ncol
        icright = icleft + ncol - 1
        spd.append([(icleft,), head_analyt[icleft]])
        spd.append([(icright,), head_analyt[icright]])
    # if domain included, specify heads along top and bottom of model
    if domain:
        for jcol in range(1, ncol - 1):
            ictoplay = jcol
            icbotlay = ictoplay + (nlay - 1) * ncol
            spd.append([(ictoplay,), head_analyt[ictoplay]])
            spd.append([(icbotlay,), head_analyt[icbotlay]])
            
    chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=spd)

    budget_file = sim_name + '.bud'
    head_file = sim_name + '.hds'
    oc = flopy.mf6.ModflowGwfoc(gwf,
                                budget_filerecord=budget_file,
                                head_filerecord=head_file,
                                saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                                printrecord=[('BUDGET', 'ALL'), ('HEAD', 'ALL')],)

    return sim

In [ ]:
def simulation(idx, silent=True):
    key = list(parameters.keys())[idx]
    params = parameters[key].copy()
    sim = build_model(key, **params)
    sim.write_simulation(silent=silent)
    success, buff = sim.run_simulation(silent=silent, report=True)
    if not success:
        print(buff)
    else:
        gwf, head, spdismf6, qx, qy, qz, qmagmid, qangmid, qnface, flow, flow_error = extract_results(idx, sim)
    return(gwf, head, spdismf6, qx, qy, qz, qmagmid, qangmid, qnface, flow, flow_error) 

In [ ]:
def recalculate_spdis(qx, qy, qz, staggered, icelltype, delz_chan, bud):
    
    # recalculate specific discharge excluding connections that cross the channel boundary
    
    flowja = bud.get_data(text='FLOW-JA')[0][0][0]
    gp = d2d.get_gridprops_disu6()
    iac = gp["iac"]
    ja = gp["ja"]
    ihc = gp["ihc"]
    topbycell = gp["top"]
    botbycell = gp["bot"]
    hwva = gp["hwva"]
    
    iconn = -1
    icell = -1
    for il in iac:
        icell += 1
        qx[icell] = 0.
        qz[icell] = 0.
        qxnumer = 0.
        qxdenom = 0.
        qznumer = 0.
        qzdenom = 0.
        for ilnbr in range(il):
            iconn += 1
            inbr = ja[iconn]
            if (inbr == icell):
                continue
            if icelltype[inbr] != icelltype[icell]:
                continue
            if ihc[iconn] > 0:
                if staggered:
                    dz = min(topbycell[icell], topbycell[inbr]) - max(botbycell[icell], botbycell[inbr])
                else:
                    dz = delz_chan
                qxincr = flowja[iconn] / (hwva[iconn] * dz)
                # equal weight given to each face, but could weight by distance instead
                if (inbr < icell):
                    qxnumer += qxincr
                else:
                    qxnumer -= qxincr
                qxdenom += 1.
            else:
                qzincr = flowja[iconn] / hwva[iconn]
                # equal weight given to each face, but could weight by distance instead
                if (inbr < icell):
                    qznumer -= qzincr
                else:
                    qznumer += qzincr
                qzdenom += 1.
        if qxdenom > 0.:
            qx[icell] = qxnumer / qxdenom
        if qzdenom > 0.:
            qz[icell] = qznumer / qzdenom
            
    return

In [ ]:
def get_face_info(d2d):
    
    import flopy.utils.gridgen as gg
    
    gp = d2d.get_gridprops_disu6()
    iac = gp["iac"]
    ja = gp["ja"]
    #nodes = gp["nodes"]
    nja = gp["nja"]
    cell2d = gp["cell2d"]
    top = gp["top"]
    bot = gp["bot"]
    cl12 = gp["cl12"]
    ia = gg.get_ia_from_iac(iac)
    isym = gg.get_isym(ia, ja)
    
    xface = np.zeros((nja))
    yface = np.zeros((nja))
    zface = np.zeros((nja))
    ifacetype = np.zeros((nja))
    iconn = -1
    for icell, il in enumerate(iac):
        xcell = cell2d[icell][1]
        ycell = cell2d[icell][2]
        zcell = 0.5 * (top[icell] + bot[icell])
        topcell = top[icell]
        botcell = bot[icell]
        for ilnbr in range(il):
            iconn += 1
            inbr = ja[iconn]
            ifacetype[iconn] = icelltype[icell] + icelltype[inbr]
            if (inbr == icell):
                ifacetype[iconn] = -1
            xnbr = cell2d[inbr][1]
            ynbr = cell2d[inbr][2]
            znbr = 0.5 * (top[inbr] + bot[inbr])
            cl1 = cl12[iconn]
            cl2 = cl12[isym[iconn]]
            frac = cl1 / (cl1 + cl2)
            omfrac = 1. - frac
            xface[iconn] = omfrac * xcell + frac * xnbr
            yface[iconn] = omfrac * ycell + frac * ynbr
            #zface[iconn] = omfrac * zcell + frac * znbr
            topnbr = top[inbr]
            botnbr = bot[inbr]
            topmin = min(topcell, topnbr)
            botmax = max(botcell, botnbr)
            zface[iconn] = 0.5 * (topmin + botmax)            

    return (xface, yface, zface, ifacetype)

In [ ]:
def normal_fluxes(staggered, icelltype, delz_chan, bud):
    
    # calculate normal fluxes (flow / area) at cell-cell interfaces
    
    flowja = bud.get_data(text='FLOW-JA')[0][0][0]
    gp = d2d.get_gridprops_disu6()
    iac = gp["iac"]
    ja = gp["ja"]
    ihc = gp["ihc"]
    topbycell = gp["top"]
    botbycell = gp["bot"]
    hwva = gp["hwva"]
    nja = gp["nja"]
    
    qnface = np.zeros((len(flowja)))
    iconn = -1
    icell = -1
    for il in iac:
        icell += 1
        for ilnbr in range(il):
            iconn += 1
            inbr = ja[iconn]
            if (inbr == icell):
                qnface[iconn] = 0.
                continue
            if ihc[iconn] > 0:
                if staggered:
                    dz = min(topbycell[icell], topbycell[inbr]) - max(botbycell[icell], botbycell[inbr])
                else:
                    dz = delz_chan
                qnface[iconn] = flowja[iconn] / (hwva[iconn] * dz)
                if (inbr > icell):
                    qnface[iconn] = - qnface[iconn]
            else:
                qnface[iconn] = flowja[iconn] / hwva[iconn]
                if (inbr < icell):
                    qnface[iconn] = - qnface[iconn]

    return (qnface)

In [ ]:
def extract_spdis_indicators(qx, qy, qz):

    nlay = mgs.nlay
    jcolmid = int(ncol / 2)
    klaymid = int(nlay / 2)
    
    qmagmin = qangmin = qmagmid = float('inf')
    qmagmax = qangmax = qangmid = float('-inf')
    qmagavg = qangavg = 0.
    ncount = 0
    #icell = -1
    #for klay in range(nlay):
    icell = nlay_dom_upper * ncol - 1
    for klay in range(nlay_dom_upper, nlay_dom_upper + nlay_chan):
        for jcol in range(ncol):
            icell += 1
            qqx = qx[icell]
            qqz = qz[icell]
            qmag = math.sqrt(qqx * qqx + qqz * qqz)
            if (qqx == 0.):
                if (qqz == 0.):
                    qang = 0.
                elif (qqz > 0.):
                    qang = 90.
                else:
                    qang = -90.
            else:
                qang = math.atan2(qqz, qqx) * 180. / math.pi
            ncount += 1
            qmagmin = min([qmagmin, qmag])
            qmagmax = max([qmagmax, qmag])
            qmagavg += qmag
            qangmin = min([qangmin, qang])
            qangmax = max([qangmax, qang])
            qangavg += qang
            if (klay == klaymid) and (jcol == jcolmid):
                qmagmid = qmag
                qangmid = qang
    qmagavg = qmagavg / ncount
    qangavg = qangavg / ncount
    qmagana = 1.
    qangana = theta

    return(qmagmid, qangmid)

In [ ]:
def extract_results(idx, sim):

    sim_name = list(parameters.keys())[idx]
    sim_ws = os.path.join(ws, sim_name)
    gwf = sim.get_model(sim_name)
    staggered = parameters[sim_name]["staggered"]
    
    simdata = [["sim_name", sim_name]]
    for key, value in parameters[sim_name].items():
        simdata.append([key, value])
    #print(tabulate(simdata, tablefmt="simple", floatfmt=".14f"))
    
    head = gwf.output.head().get_data()
    
    bud = gwf.output.budget()
    spdismf6 = bud.get_data(text='DATA-SPDIS')[0]
    chdbud = bud.get_data(text='CHD')[0]
    qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdismf6, gwf)
    # optionally recalculate specific discharge
    if spdis_recalc:
        recalculate_spdis(qx, qy, qz, staggered, icelltype, delz_chan, bud)
    # extract specific discharge indicators
    qmagmid, qangmid = extract_spdis_indicators(qx, qy, qz)
    # calculate normal fluxes at faces
    qnface = normal_fluxes(staggered, icelltype, delz_chan, bud)
    flow = volflow_1(qx, qy, qz, thick_arr)
    #flow = volflow_2(chdbud)
    flow_error = (flow - flow_analytical) / flow_analytical
    return(gwf, head, spdismf6, qx, qy, qz, qmagmid, qangmid, qnface, flow, flow_error)

In [ ]:
# Average volumetric flow - Integrate cell volume x spdiscell / length channel (parallelogram, use midline)

def volflow_1(qx, qy, qz, thick_arr):

    qmag = np.ones_like(qx)
    for i in range(len(qx)):
        qmag[i] = (qx[i]**2 + qy[i]**2 + qz[i]**2)**0.5
           
    Q = thick_arr.flatten() * np.multiply(qmag, icelltype)
    L = Lx/np.cos(np.radians(theta))
    vol_flow = Q.sum() / L
    
    return(vol_flow)

def volflow_2(chdbud):

    Q = 0.
    for j in range(len(chdbud)):
        
        if chdbud[j][2]>0:   # we are querying if flow is positive
            Q+= chdbud[j][2] # then if it is we add it to our flow value    
    return(Q)